In [ ]:
import cv2
import json
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt

In this notebook, we will try to estimate the 3D coordinates of an extrapolated keypoint located on the lateral side of the fish. The extrapolated keypoint will be the part of the fish that appears as the midpoint of the dorsal fin coordinates and the pelvic fic coordinates in the left iamge. We will use prior knowledge of fish dimensions to determine the disparity bounds on this point. This means that we can constrain the search for the right image correspondency to a patch based on these disparity bounds.

<h1> Load a sample stereo image and associated image keypoint coordinates </h1>

In [ ]:
epoch = 1549544491230
left_image_path = '/Users/aloksaxena/Documents/aquabyteai/repos/cv_research/alok/notebooks/rectification/data/gtsf_a/rectified_images/left_small-pen-test-site_1_{}.jpg'.format(epoch)
right_image_path = '/Users/aloksaxena/Documents/aquabyteai/repos/cv_research/alok/notebooks/rectification/data/gtsf_a/rectified_images/right_small-pen-test-site_1_{}.jpg'.format(epoch)

left_image = cv2.imread(left_image_path)
right_image = cv2.imread(right_image_path)

In [ ]:
plt.figure(figsize=(50, 50))
plt.imshow(left_image)

<h1> Get keypoint world coordinates and 2D left and right image coordinates </h1>

In [ ]:
keypoint_data = json.load(open('/Users/aloksaxena/Documents/aquabyteai/repos/cv_research/alok/notebooks/playground/keypoint_data.json'))

In [ ]:
left_kp_data_json = [kp for kp in keypoint_data if kp['External ID'] == 'left_small-pen-test-site_1_{}.jpg'.format(epoch)][0]
right_kp_data_json = [kp for kp in keypoint_data if kp['External ID'] == 'right_small-pen-test-site_1_{}.jpg'.format(epoch)][0]

In [ ]:
left_kp_data = {}
right_kp_data = {}
body_parts = list(left_kp_data_json['Label'].keys())
for body_part in body_parts:
    left_kp_dict = left_kp_data_json['Label'][body_part][0]['geometry']
    left_kp = np.array([left_kp_dict['x'], left_kp_dict['y']])
    
    right_kp_dict = right_kp_data_json['Label'][body_part][0]['geometry']
    right_kp = np.array([right_kp_dict['x'], right_kp_dict['y']])
    left_kp_data[body_part] = left_kp
    right_kp_data[body_part] = right_kp
    

<h1> Compute the 3D world coordinates of the keypoints </h1>

In [ ]:
# DEFINE OPTICAL PROPERTIES

# all distance are in meters
FOCAL_LENGTH = 0.0085
BASELINE = 0.1044
PIXEL_SIZE_M = 3.45 * 1e-6
FOCAL_LENGTH_PIXEL = FOCAL_LENGTH / PIXEL_SIZE_M
IMAGE_SENSOR_WIDTH = 0.01412
IMAGE_SENSOR_HEIGHT = 0.01035
PIXEL_COUNT_WIDTH = 4096
PIXEL_COUNT_HEIGHT = 3000
CHECKERBOARD_SIDE_LENGTH = 0.0495



In [ ]:
def convert_to_world_point(x, y, d):
    """ from pixel coordinates to world coordinates """
    
    image_center_x = PIXEL_COUNT_WIDTH / 2.0  
    image_center_y = PIXEL_COUNT_HEIGHT / 2.0
    px_x = x - image_center_x
    px_z = image_center_y - y

    sensor_x = px_x * (IMAGE_SENSOR_WIDTH / PIXEL_COUNT_WIDTH)
    sensor_z = px_z * (IMAGE_SENSOR_HEIGHT / PIXEL_COUNT_HEIGHT)

    # d = depth_map[y, x]
    world_y = d
    world_x = (world_y * sensor_x) / FOCAL_LENGTH
    world_z = (world_y * sensor_z) / FOCAL_LENGTH
    return np.array([world_x, world_y, world_z])



def depth_from_disp(disp):
    """ calculate the depth of the point based on the disparity value """
    depth = FOCAL_LENGTH_PIXEL*BASELINE / np.array(disp)
    return depth

def disp_from_depth(depth):
    disp = FOCAL_LENGTH_PIXEL * BASELINE / depth
    return disp


In [ ]:
world_kp_data = {}
for body_part in body_parts:
    left_kp = left_kp_data[body_part]
    right_kp = right_kp_data[body_part]
    
    disp = abs(left_kp[0] - right_kp[0])
    depth = depth_from_disp(disp)
    world_kp = convert_to_world_point(left_kp[0], left_kp[1], depth)
    world_kp_data[body_part] = world_kp
    
    
    

<h1> Get extrapolated keypoint coordinates in left image, and associated disparity bounds for determining the corresponding pixel in the right image </h1>

In [ ]:
bp_1, bp_2 = 'upper lip', 'tail: notch'

left_extrap_kp = (0.5 * left_kp_data[bp_1] + 0.5 * left_kp_data[bp_2]).astype('int64')
bp_1_depth = world_kp_data[bp_1][1]
bp_2_depth = world_kp_data[bp_2][1]

# need to determine lower and upper bounds here in a data driven fashion from GTSF data
extrap_kp_max_depth = (bp_1_depth + bp_2_depth) / 2.0 - 0.02
extrap_kp_min_depth = (bp_1_depth + bp_2_depth) / 2.0 - 0.1

extrap_kp_min_disp = disp_from_depth(extrap_kp_max_depth)
extrap_kp_max_disp = disp_from_depth(extrap_kp_min_depth)


<h1> Compute the feature descriptor for the extrapolated keypoint in the left image </h1>

In [ ]:
left_window_size = 100
left_box = left_image[left_extrap_kp[1]-left_window_size//2:left_extrap_kp[1]+left_window_size//2, 
                      left_extrap_kp[0]-left_window_size//2:left_extrap_kp[0]+left_window_size//2]
right_box = right_image[left_extrap_kp[1]-left_window_size//2:left_extrap_kp[1]+left_window_size//2,
                        left_extrap_kp[0]-int(extrap_kp_max_disp)-left_window_size//2:left_extrap_kp[0]-int(extrap_kp_min_disp)+left_window_size//2]


In [ ]:
plt.imshow(left_box)

In [ ]:
plt.imshow(right_box)

In [ ]:
orb = cv2.ORB_create()
kp1, des1 = orb.detectAndCompute(left_box,None)
kp2, des2 = orb.detectAndCompute(right_box,None)

In [ ]:
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = bf.match(des1,des2)
matches = sorted(matches, key = lambda x:x.distance)
img3 = cv2.drawMatches(left_box,kp1,right_box,kp2,matches[:5], None, flags=2)
plt.figure(figsize=(50, 50))
plt.imshow(img3),plt.show()

In [ ]:
matches[0].distance